In [ ]:
!pip install -r requirements.txt
!pip install scikit-image
!pip install deepface

In [2]:

from skimage.metrics import structural_similarity as compare_ssim
import os
import cv2
import numpy as np
import warnings
import time
import math
import torch
from deepface import DeepFace
from facenet_pytorch import MTCNN
from src.anti_spoof_predict import AntiSpoofPredict
from src.generate_patches import CropImage
from src.utility import parse_model_name
warnings.filterwarnings('ignore')


In [5]:
device = torch.device("cpu")

# Create the model
mtcnn = MTCNN(keep_all=True, device=device)

In [15]:
###implement fps limiter
fps_limit = 10
frame_number = 0
start_time = 0
cap= cv2.VideoCapture("/home/sidk_1023/Facial_Recognition_Pipeline-main/face-demographics-walking-and-pause.mp4")


In [7]:
framediff_pass = True

In [16]:
class FrameDif:
    def __init__(self,threshold):
        self.threshold = threshold
    def ssim(self,curr_time,prev_time):
        cap.set(cv2.CAP_PROP_POS_MSEC,curr_time) 
        ret,curr_frame = cap.read()
        cv2.imshow('curr_frame',curr_frame)
        #curr_frame = cv2.resize(curr_frame, (600, 400))
        cap.set(cv2.CAP_PROP_POS_MSEC,prev_time) 
        ret,prev_frame = cap.read()
        #prev_frame = cv2.resize(prev_frame, (600, 400))
        current_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        previous_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        (score,frame_diff) = compare_ssim(previous_frame_gray,current_frame_gray,full = True)
        if (score< self.threshold):
            framediff_pass = False
            return(curr_time)
        return(-1)
        

In [18]:
framedif = FrameDif(0.99)
tcurr = 10000
tprev = 1000
val = framedif.ssim(tcurr,tprev)
val

10000

In [22]:
tcurr = 10000
tprev =10000
framediff_flag = True
while(framediff_flag):
    timestamp =framedif.ssim(tcurr,tprev)
    tprev = tcurr 
    tcurr = tcurr+1
    if timestamp>0:
        framediff_flag =False
    
print('I am out of loop '+ str(tcurr))
    

I am out of loop 10043


In [68]:
tcurr

2000

In [14]:
#framediff_flag
cap.set(cv2.CAP_PROP_POS_MSEC, 10000 )
ret, frame = cap.read()
cv2.imshow("Frame", frame)
if ret:
    #cv2.imshow("Frame", frame)
    boxes, conf = mtcnn.detect(frame)
    # conf is a list of the confidence values of each detection
    # box is a list of four tuples where each of the tuples
    # contain the x,y,width,height of a box that contains the detected face

    #    print('\n boxes: ', boxes)
    #    print('\n conf: ', conf)
    # Define a confidence threshold:
    confidence_threshold = 0.95
    multiple_faces_detected = False
    no_face_detected = False
    n_faces = 0

    for i in conf:

        if i == None: boxes, conf = mtcnn.detect(frame)
    # conf is a list of the confidence values of each detection
    # box is a list of four tuples where each of the tuples
    # contain the x,y,width,height of a box that contains the detected face

    #    print('\n boxes: ', boxes)
    #    print('\n conf: ', conf)
    # Define a confidence threshold:
    confidence_threshold = 0.95
    multiple_faces_detected = False
    no_face_detected = False
    n_faces = 0

    for i in conf:

        if i == None:
            i = 0
        if i > confidence_threshold:
            n_faces += 1

            i = 0
        if i > confidence_threshold:
            n_faces += 1

    if n_faces > 1:
        multiple_faces = True
    else:
        multiple_faces = False

    if n_faces == 0:
        no_face_detected = True

    if multiple_faces == True:
        print(
            str(n_faces)
            + " faces detected! "
            + "for frame: "
            + str(frame_number)
            + " whose timestamp is: ",
            str(cap.get(cv2.CAP_PROP_POS_MSEC)),
        )

    if no_face_detected == True:
        print(
            "No face detected!"
            + "for frame: "
            + str(frame_number)
            + " whose timestamp is: ",
            str(cap.get(cv2.CAP_PROP_POS_MSEC)),
        )

    frame_number += 1

    n_faces_str = "No. of faces = " + str(n_faces)
    frame_number_str = "Frame No. = " + str(frame_number)
    time_stamp_str = "Timestamp = " + \
        str(cap.get(cv2.CAP_PROP_POS_MSEC))

    if conf[0] != None:
        for (x, y, w, h) in boxes:
            text = f"{conf[0]*100:.2f}%"
            x, y, w, h = int(x), int(y), int(w), int(h)

            cv2.putText(
                frame,
                text,
                (x, y - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
            )

            cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 1)

    cv2.putText(
        frame,
        frame_number_str,
        (0, 15),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        1,
    )

    cv2.putText(
        frame,
        time_stamp_str,
        (0, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        1,
    )

    cv2.putText(
        frame,
        n_faces_str,
        (0, 45),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 255, 0),
        1,
    )
#
# cv2.imshow("Frame", frame)
    
# if cv2.waitKey(25) & 0xFF == ord("q"):cap
#     break

